# Set constants

In [1]:
test_data_path = "./tmp/test.csv"

# Load test data

In [2]:
import yfinance as yf
import pandas as pd

In [3]:
if 1 == 0:
    data_df:pd.DataFrame =  yf.Ticker("EURUSD=X").history(start="2023-01-01",end="2024-01-01", interval="1d")
    data_df.index = data_df.index.date
    data_df.index.name = "Date"
    data_df["High"] = data_df[["Open","High","Low","Close"]].max(axis=1)
    data_df["Low"] = data_df[["Open","High","Low","Close"]].min(axis=1)
    data_df["Volume"] = 10000000
    data_df.to_csv(test_data_path)

In [4]:
data_df:pd.DataFrame = pd.read_csv(test_data_path)
data_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-01-02,1.070973,1.071237,1.065326,1.070973,10000000,0.0,0.0
1,2023-01-03,1.067771,1.068262,1.052155,1.067771,10000000,0.0,0.0
2,2023-01-04,1.054685,1.063151,1.054596,1.054685,10000000,0.0,0.0
3,2023-01-05,1.060637,1.063264,1.051558,1.060637,10000000,0.0,0.0
4,2023-01-06,1.052222,1.062225,1.048526,1.052222,10000000,0.0,0.0
...,...,...,...,...,...,...,...,...
255,2023-12-25,1.102657,1.104240,1.099989,1.102657,10000000,0.0,0.0
256,2023-12-26,1.102026,1.103997,1.100958,1.102026,10000000,0.0,0.0
257,2023-12-27,1.104301,1.112248,1.102925,1.104301,10000000,0.0,0.0
258,2023-12-28,1.110864,1.113945,1.107101,1.110864,10000000,0.0,0.0


# Set strategy

In [5]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import quandlfeed
from pyalgotrade.technical import ma
from pyalgotrade.broker.backtesting import Broker
from pyalgotrade.bar import BasicBar
from pyalgotrade.strategy.position import LongPosition, Position, ShortPosition
from pyalgotrade.broker import OrderExecutionInfo, Order
from src.simulation.models import Deal,List

In [6]:
class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, signal_delta:float = 0):
        super(MyStrategy, self).__init__(feed)
        # We want a 15 period SMA over the closing prices.
        self.__sma = ma.SMA(feed[instrument].getCloseDataSeries(), 15)
        self.__instrument = instrument
        self.__position:Position = None
        self.cap_log = {}
        self.__last_deal:Deal = None
        self.deal_list:List[Deal] = []
        self.__signal = signal_delta

    def _action_to_stirng(self, action):
        if action == Order.Action.BUY:
            return "BUY"
        elif action == Order.Action.SELL:
            return "SELL"
        elif action == Order.Action.SELL_SHORT:
            return "SELL_SHORT"
        elif action == Order.Action.BUY_TO_COVER:
            return "BUY_TO_COVER"
        else:
            raise Exception(f"Unexpected action {action}")

    def onEnterOk(self, position:Position):
        order: Order = position.getEntryOrder()
        execInfo: OrderExecutionInfo = order.getExecutionInfo()

        self.info(f"{self._action_to_stirng(order.getAction())} at {execInfo.getPrice()}")
        self.__last_deal = Deal.BuildFromCap(execInfo.getDateTime(), execInfo.getPrice(), execInfo.getQuantity(), order.getInstrument(), self.__get_cur_equity())
        self.deal_list.append(self.__last_deal)

    def onEnterCanceled(self, position:Position):
        self.__position = None

    def onExitOk(self, position:Position):
        order: Order = position.getExitOrder()
        execInfo: OrderExecutionInfo = order.getExecutionInfo()
        self.info(f"Close {self._action_to_stirng(order.getAction())} order at {execInfo.getPrice()}")
        self.__position = None
        self.__last_deal.close_deal(execInfo.getDateTime(), execInfo.getPrice())
        self.__last_deal = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def __get_cur_equity(self)->float:
        return self.getBroker().getEquity()

    def onBars(self, bars):
        equity = self.__get_cur_equity()

        bar: BasicBar = bars[self.__instrument]
        
        self.cap_log[bar.getDateTime()] = equity

        if self.__sma[-1] is None:
            return
        
        
        if bar.getPrice()/self.__sma[-1] - 1 > self.__signal:
            if self.__position is None:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, 100000, True)
            elif isinstance(self.__position, ShortPosition) and not self.__position.exitActive():
                self.__position.exitMarket()
        elif self.__sma[-1]/bar.getPrice() - 1 > self.__signal:
            if self.__position is None:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterShort(self.__instrument, 100000, True)
            elif isinstance(self.__position, LongPosition) and not self.__position.exitActive():
                self.__position.exitMarket()


In [7]:

# Load the bar feed from the CSV file
feed = quandlfeed.Feed()
feed.addBarsFromCSV("orcl", test_data_path)

# Evaluate the strategy with the feed's bars.
myStrategy = MyStrategy(feed, "orcl")
myStrategy.run()

2023-01-23 00:00:00 strategy [INFO] BUY at 1.0864723920822144
2023-02-07 00:00:00 strategy [INFO] Close SELL order at 1.0730764865875244
2023-02-08 00:00:00 strategy [INFO] SELL_SHORT at 1.0729728937149048
2023-03-03 00:00:00 strategy [INFO] Close BUY_TO_COVER order at 1.0599732398986816
2023-03-06 00:00:00 strategy [INFO] SELL_SHORT at 1.0626201629638672
2023-03-08 00:00:00 strategy [INFO] Close BUY_TO_COVER order at 1.05507493019104
2023-03-09 00:00:00 strategy [INFO] SELL_SHORT at 1.0549525022506714
2023-03-14 00:00:00 strategy [INFO] Close BUY_TO_COVER order at 1.0725010633468628
2023-03-15 00:00:00 strategy [INFO] BUY at 1.0727657079696655
2023-03-17 00:00:00 strategy [INFO] Close SELL order at 1.0614134073257446
2023-03-20 00:00:00 strategy [INFO] SELL_SHORT at 1.0678855180740356
2023-03-21 00:00:00 strategy [INFO] Close BUY_TO_COVER order at 1.071914792060852
2023-03-22 00:00:00 strategy [INFO] BUY at 1.0774236917495728
2023-05-03 00:00:00 strategy [INFO] Close SELL order at 1.1

In [8]:
myStrategy.getBroker().getShares("orcl")

100000

In [9]:
myStrategy.getBroker().getEquity()

1006301.6891479492

# Define trading simulator

In [10]:
from src.simulation.abs_trading_simulatior import absTradingSimulatior,StrategyId,SimulationConfig,SimulationLog


In [11]:
from pyalgotrade.barfeed.csvfeed import BarFeed, bar, GenericRowParser
from pyalgotrade.bar import Frequency
class DataFrameBarFeed(BarFeed):
    """A BarFeed that loads bars from CSV files that have the following format:
    ::

        Date Time,Open,High,Low,Close,Volume,Adj Close
        2013-01-01 13:59:00,13.51001,13.56,13.51,13.56,273.88014126,13.51001

    :param frequency: The frequency of the bars. Check :class:`pyalgotrade.bar.Frequency`.
    :param timezone: The default timezone to use to localize bars. Check :mod:`pyalgotrade.marketsession`.
    :type timezone: A pytz timezone.
    :param maxLen: The maximum number of values that the :class:`pyalgotrade.dataseries.bards.BarDataSeries` will hold.
        Once a bounded length is full, when new items are added, a corresponding number of items are discarded from the
        opposite end. If None then dataseries.DEFAULT_MAX_LEN is used.
    :type maxLen: int.

    .. note::
        * The CSV file **must** have the column names in the first row.
        * It is ok if the **Adj Close** column is empty.
        * When working with multiple instruments:

         * If all the instruments loaded are in the same timezone, then the timezone parameter may not be specified.
         * If any of the instruments loaded are in different timezones, then the timezone parameter should be set.
    """

    def __init__(self, frequency:Frequency, timezone=None, maxLen=None):
        super(DataFrameBarFeed, self).__init__(frequency, maxLen)
        self.__timezone = timezone
        # Assume bars don't have adjusted close. This will be set to True after
        # loading the first file if the adj_close column is there.
        self.__haveAdjClose = False

        self.__barClass = bar.BasicBar

        self.__dateTimeFormat = "%Y-%m-%d"
        self.__columnNames = {
            "datetime": "Date",
            "open": "Open",
            "high": "High",
            "low": "Low",
            "close": "Close",
            "volume": "Volume",
            "adj_close": "Adj Close",
        }
        # self.__dateTimeFormat expects time to be set so there is no need to
        # fix time.
        self.setDailyBarTime(None)

    def barsHaveAdjClose(self):
        return self.__haveAdjClose

    def setNoAdjClose(self):
        self.__columnNames["adj_close"] = None
        self.__haveAdjClose = False

    def setColumnName(self, col, name):
        self.__columnNames[col] = name

    def setDateTimeFormat(self, dateTimeFormat):
        """
        Set the format string to use with strptime to parse datetime column.
        """
        self.__dateTimeFormat = dateTimeFormat

    def setBarClass(self, barClass):
        self.__barClass = barClass

    def addBarsFromDataFrame(self, instrument: str, dataframe: pd.DataFrame, timezone=None, skipMalformedBars=False):
        """Load bars from DataFrame

        Args:
            instrument (str): Instrument identifier
            dataframe (pandas.DataFrame): loaded dataframe
            timezone (pytz timezone, optional): The timezone to use to localize bars. Check :mod:`pyalgotrade.marketsession`. Defaults to None.
            skipMalformedBars (bool, optional): True to skip errors while parsing bars. Defaults to False.
        """
        if timezone is None:
            timezone = self.__timezone

        rowParser = GenericRowParser(
            self.__columnNames, self.__dateTimeFormat, self.getDailyBarTime(), self.getFrequency(),
            timezone, self.__barClass
        )

        def parse_bar_skip_malformed(row):
            ret = None
            try:
                ret = rowParser.parseBar(row)
            except Exception:
                pass
            return ret

        if skipMalformedBars:
            parse_bar = parse_bar_skip_malformed
        else:
            parse_bar = rowParser.parseBar

        loadedBars = []
        for row in dataframe.to_dict("records"):
            row[self.__columnNames["datetime"]] = str(
                row[self.__columnNames["datetime"]])
            bar_ = parse_bar(row)
            if bar_ is not None and (self.getBarFilter() is None or self.__barFilter.includeBar(bar_)):
                loadedBars.append(bar_)

        self.addBarsFromSequence(instrument, loadedBars)

        if rowParser.barsHaveAdjClose():
            self.__haveAdjClose = True
        elif self.__haveAdjClose:
            raise Exception(
                "Previous bars had adjusted close and these ones don't have.")



In [12]:
class TradingSimulatior(absTradingSimulatior):
    @property
    def strategy_id(self)->StrategyId:
        return StrategyId("Example Strategy", "v1")
    
    def _run(self, run_config: SimulationConfig)->SimulationLog:
        # Load the bar feed from the CSV file
        
        feed = DataFrameBarFeed(Frequency.DAY)
        feed.addBarsFromDataFrame("EURUSD",run_config.period.filter_df(data_df))

        signal_delta = run_config.strategy_cfg["delta"]
        # Evaluate the strategy with the feed's bars.
        myStrategy = MyStrategy(feed, "EURUSD",signal_delta)
        myStrategy.run()
        return SimulationLog(myStrategy.cap_log,myStrategy.deal_list)

# Optimizing

In [13]:
from src.optimization_analyzer.optimization_analyzer import OptimizationAnalyzer, OptimizationConfig
from NNTrade.common import TimeFrame

In [14]:
from datetime import date
from src.common.candle_config import CandleConfig
from src.common.candle_data_set_config import CandleDataSetConfig
from src.common.date_period import DatePeriod
from src.optimization.config.strategy_config_sets import StrategyConfigSet


simulation_report_factory: absTradingSimulatior

opt_config = OptimizationConfig(CandleDataSetConfig.BuildFrom(CandleConfig("EURUSD"),TimeFrame.D), DatePeriod(date(2023,1,1), date(2024,1,1)),StrategyConfigSet({"delta":[0,0.0001,0.0005, 0.0010, 0.005, 0.01,]}))

In [15]:
opt_analyzer = OptimizationAnalyzer(TradingSimulatior())
opt_analyzer.analis_optimization_flow(opt_config,False)

2024-02-15 00:25:35,946 SingleAnalizator [INFO] Define analization rounds intervals
2024-02-15 00:25:35,947 DefaultPeriodSplitter [INFO] Splitting {'from': datetime.date(2023, 1, 1), 'untill': datetime.date(2024, 1, 1)} on optmization and forward intervals
2024-02-15 00:25:35,948 DefaultPeriodSplitter [INFO] Splitted on [{'optimization': {'from': datetime.date(2023, 1, 1), 'untill': datetime.date(2023, 6, 30)}, 'forward': {'from': datetime.date(2023, 6, 30), 'untill': datetime.date(2023, 7, 30)}}, {'optimization': {'from': datetime.date(2023, 1, 31), 'untill': datetime.date(2023, 7, 30)}, 'forward': {'from': datetime.date(2023, 7, 30), 'untill': datetime.date(2023, 8, 29)}}, {'optimization': {'from': datetime.date(2023, 3, 2), 'untill': datetime.date(2023, 8, 29)}, 'forward': {'from': datetime.date(2023, 8, 29), 'untill': datetime.date(2023, 9, 28)}}, {'optimization': {'from': datetime.date(2023, 4, 1), 'untill': datetime.date(2023, 9, 28)}, 'forward': {'from': datetime.date(2023, 9, 2

[AnalyzationReport(optimization={'strategy_id': {'name': 'Example Strategy', 'v': 'v1'}, 'run_config': {'candle_data_set': {'stocks': {'default': {'ticker': 'EURUSD', 'timeframe': ''}}, 'step_timeframe': 'D'}, 'period': {'from': datetime.date(2023, 1, 1), 'untill': datetime.date(2023, 6, 30)}, 'strategy_cfg': {'delta': 0.001}}, 'metric': {'capital': {'yield': 0.0027695298194885254, 'yield/year': 0.005623984077963495, 'max_yield': 0.0035374999046324795, 'max_fall': -0.004222706445691167}, 'deals': {'deal_count': 8, 'success_deal_count': 2, 'fail_deal_count': 5, 'avg_net_profit': -2599.8482639535932, 'avg_net_income': 21897.22307990549, 'avg_net_loss': -12398.676801497226, 'PROM': 4.22500326713334}}, 'abs_cap_log': {datetime.datetime(2023, 1, 2, 0, 0): 1000000, datetime.datetime(2023, 1, 3, 0, 0): 1000000, datetime.datetime(2023, 1, 4, 0, 0): 1000000, datetime.datetime(2023, 1, 5, 0, 0): 1000000, datetime.datetime(2023, 1, 6, 0, 0): 1000000, datetime.datetime(2023, 1, 9, 0, 0): 1000000, 